In [2]:
import findspark
findspark.init()
findspark.find()

'C:\\spark-3.2.1-bin-hadoop3.2'

In [2]:
!pip3 install pyarrow

In [3]:
import duckdb
import numpy as np
import pandas as pd
import pyspark
from pyspark.shell import spark
import pyarrow
#from pathlib import Path
from sklearn.model_selection import cross_val_score, StratifiedKFold, train_test_split
#from sklearn.feature_extraction.text import TfidfVectorizer
#from sklearn.pipeline import Pipeline
#from sklearn.metrics import f1_score, confusion_matrix, classification_report
from matplotlib import pyplot as plt
%config InlineBackend.figure_format = 'retina'  # sharper plots
from pyspark.sql import functions as func
from pyspark.sql.types import ArrayType, FloatType, DecimalType, StringType, IntegerType
from pyspark.sql.functions import size
from pyspark.storagelevel import StorageLevel
from pyspark import SparkContext
from pyspark.sql.functions import col,when
from pyspark.sql import SparkSession
import os
 
# Optimizing Spark Conversion to Pandas https://arrow.apache.org/blog/2017/07/26/spark-arrow/
spark.conf.set("spark.sql.execution.arrow.enabled", "true")
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", 1024)
%pip show pandas

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 3.2.1
      /_/

Using Python version 3.7.11 (default, Jul 27 2021 09:42:29)
Spark context Web UI available at http://host.docker.internal:4040
Spark context available as 'sc' (master = local[*], app id = local-1647733018464).
SparkSession available as 'spark'.
Name: pandas
Version: 1.3.4
Summary: Powerful data structures for data analysis, time series, and statistics
Home-page: https://pandas.pydata.org
Author: The Pandas Development Team
Author-email: pandas-dev@python.org
License: BSD-3-Clause
Location: c:\users\anton\anaconda3\envs\tensorflowkeras\lib\site-packages
Requires: numpy, pytz, python-dateutil
Required-by: seaborn, Mesa, ktrain, codecarbon
Note: you may need to restart the kernel to use updated packages.


Reading Data

In [5]:
#Create PySpark SparkSession
spark = SparkSession.builder \
    .master() \
    .appName() \
    .getOrCreate()

In [ ]:
#Connecting with database
con = duckdb.connect(database='my-db.duckdb', read_only=False)

In [7]:
#First way
#getting dataframes and turning them into spark dfs
df1=spark.createDataFrame(con.execute("SELECT * FROM train_1").fetchdf())
df2=spark.createDataFrame(con.execute("SELECT * FROM train_2").fetchdf())
df3=spark.createDataFrame(con.execute("SELECT * FROM train_3").fetchdf())
df4=spark.createDataFrame(con.execute("SELECT * FROM train_4").fetchdf())
df5=spark.createDataFrame(con.execute("SELECT * FROM train_5").fetchdf())
df6=spark.createDataFrame(con.execute("SELECT * FROM train_6").fetchdf())
df7=spark.createDataFrame(con.execute("SELECT * FROM train_7").fetchdf())
df8=spark.createDataFrame(con.execute("SELECT * FROM train_8").fetchdf())
dfw=spark.createDataFrame(con.execute("SELECT * FROM writing").fetchdf())
dft=spark.createDataFrame(con.execute("SELECT * FROM test_hidden").fetchdf())
dfv=spark.createDataFrame(con.execute("SELECT * FROM validation_hidden").fetchdf())

In [63]:
#Second way
#first stacking/concating the dataframes and then turn them into spark df 
#(dont know which case is better 1 or 2)
df01 = con.execute("SELECT * FROM train_1").fetchdf()
df02 = con.execute("SELECT * FROM train_2").fetchdf()
df03 = con.execute("SELECT * FROM train_3").fetchdf()
df04 = con.execute("SELECT * FROM train_4").fetchdf()
df05 = con.execute("SELECT * FROM train_5").fetchdf()
df06 = con.execute("SELECT * FROM train_6").fetchdf()
df07 = con.execute("SELECT * FROM train_7").fetchdf()
df08 = con.execute("SELECT * FROM train_8").fetchdf()

df0 = pd.concat([df01,df02,df03,df04,df05,df06,df07,df08], axis = 0)
#sdf = spark.createDataFrame(df0) probably I need to provide my own schema the problem is different dtypes in orginalTitle
#we need to process them first and then concat them
df0
#should also merge writing df on tconst

,column0,tconst,primaryTitle,originalTitle,startYear,endYear,runtimeMinutes,numVotes,label
0,4,tt0010600,The Doll,Die Puppe,1919,\N,66,1898.0,True
1,7,tt0011841,Way Down East,Way Down East,1920,\N,145,5376.0,True
2,9,tt0012494,Déstiny,Der müde Tod,1921,\N,97,5842.0,True
3,25,tt0015163,The Navigator,The Navigator,1924,\N,59,9652.0,True
4,38,tt0016220,The Phantom of the Opera,The Phantom of the Opera,1925,\N,93,17887.0,True
...,...,...,...,...,...,...,...,...,...
988,9966,tt9625664,Trauma Center,NaN,2019,\N,87,12951.0,False
989,9981,tt9741310,Slaxx,Slaxx,2020,\N,77,2464.0,False
990,9982,tt9742392,Kindred,Kindred,2020,\N,101,1719.0,False
991,9996,tt9850386,The Bee Gees: How Can You Mend a Broken Heart,NaN,2020,\N,111,4144.0,True


In [98]:
df0.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7959 entries, 0 to 992
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   column0         7959 non-null   int32  
 1   tconst          7959 non-null   object 
 2   primaryTitle    7959 non-null   object 
 3   originalTitle   3971 non-null   object 
 4   startYear       7959 non-null   object 
 5   endYear         7959 non-null   object 
 6   runtimeMinutes  7959 non-null   object 
 7   numVotes        7169 non-null   float64
 8   label           7959 non-null   bool   
dtypes: bool(1), float64(1), int32(1), object(6)
memory usage: 794.3+ KB


In [90]:
df0['primaryTitle'][df0["label"]==True].count()

3990

In [91]:
df0['primaryTitle'][df0["label"]==False].count() #balanced data

3969

In [104]:
#changing to numeric to be able to check correlation and to be able to turn the whole df to spark_df
df0["runtimeMinutes"] = pd.to_numeric(df0["runtimeMinutes"],downcast='integer', errors = 'coerce')


#correlation between columns providing info on what is usefull and what is not
#will perform this check for every new feature
df0.corr(method = 'spearman')

,column0,runtimeMinutes,numVotes,label
column0,1.000000,0.058717,-0.063430,-0.201733
runtimeMinutes,0.058717,1.000000,0.212816,0.335209
numVotes,-0.063430,0.212816,1.000000,0.225713
label,-0.201733,0.335209,0.225713,1.000000


In [105]:
df0.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7959 entries, 0 to 992
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   column0         7959 non-null   int32  
 1   tconst          7959 non-null   object 
 2   primaryTitle    7959 non-null   object 
 3   originalTitle   3971 non-null   object 
 4   startYear       7959 non-null   object 
 5   endYear         7959 non-null   object 
 6   runtimeMinutes  7946 non-null   float64
 7   numVotes        7169 non-null   float64
 8   label           7959 non-null   bool   
dtypes: bool(1), float64(2), int32(1), object(5)
memory usage: 794.3+ KB


In [109]:
df00 = spark.createDataFrame(df0)
df00.show(30)

+-------+---------+--------------------+--------------------+---------+-------+--------------+--------+-----+
|column0|   tconst|        primaryTitle|       originalTitle|startYear|endYear|runtimeMinutes|numVotes|label|
+-------+---------+--------------------+--------------------+---------+-------+--------------+--------+-----+
|      4|tt0010600|            The Doll|           Die Puppe|     1919|     \N|          66.0|  1898.0| true|
|      7|tt0011841|       Way Down East|       Way Down East|     1920|     \N|         145.0|  5376.0| true|
|      9|tt0012494|             Déstiny|        Der müde Tod|     1921|     \N|          97.0|  5842.0| true|
|     25|tt0015163|       The Navigator|       The Navigator|     1924|     \N|          59.0|  9652.0| true|
|     38|tt0016220|The Phantom of th...|The Phantom of th...|     1925|     \N|          93.0| 17887.0| true|
|     42|tt0016630|     Báttling Bútlér|     Battling Butler|     1926|     \N|          77.0|  3285.0| true|
|     81|t

In [47]:
df.columns

['_c0',
 'tconst',
 'primaryTitle',
 'originalTitle',
 'startYear',
 'endYear',
 'runtimeMinutes',
 'numVotes',
 'label']

In [33]:
df1 = df1.na.drop(how = "any",subset=["numVotes"])
# Doesnt make sense to build a ratings model without having the number
# of votes available, reduces interpretability as well, so we remove rows without numVotes


#We will have to check if we loose too much information on other features though
#we could replace it with the avg num_vote, if it makes sense, but still I m not sure 

In [110]:
df00 = df00.withColumn("startYear", \
       when(col("startYear")=="\\N" ,col("endYear")) \
          .otherwise(col("startYear"))) 
# This is a mistake, so when there is no startYear available we replace it with endYear
df00.toPandas().head()

,column0,tconst,primaryTitle,originalTitle,startYear,endYear,runtimeMinutes,numVotes,label
0,4,tt0010600,The Doll,Die Puppe,1919,\N,66.0,1898.0,True
1,7,tt0011841,Way Down East,Way Down East,1920,\N,145.0,5376.0,True
2,9,tt0012494,Déstiny,Der müde Tod,1921,\N,97.0,5842.0,True
3,25,tt0015163,The Navigator,The Navigator,1924,\N,59.0,9652.0,True
4,38,tt0016220,The Phantom of the Opera,The Phantom of the Opera,1925,\N,93.0,17887.0,True


In [111]:
df00 = df00.withColumn("originalTitle", \
       when(col("originalTitle")==col("primaryTitle") ,"null") \
          .otherwise(col("originalTitle"))) 
# When there is a similar original and primary title, remove the original title
# since original title is reserved for non-English films and it adds noise to our data
df00.toPandas()

,column0,tconst,primaryTitle,originalTitle,startYear,endYear,runtimeMinutes,numVotes,label
0,4,tt0010600,The Doll,Die Puppe,1919,\N,66.0,1898.0,True
1,7,tt0011841,Way Down East,null,1920,\N,145.0,5376.0,True
2,9,tt0012494,Déstiny,Der müde Tod,1921,\N,97.0,5842.0,True
3,25,tt0015163,The Navigator,null,1924,\N,59.0,9652.0,True
4,38,tt0016220,The Phantom of the Opera,null,1925,\N,93.0,17887.0,True
...,...,...,...,...,...,...,...,...,...
7954,9966,tt9625664,Trauma Center,None,2019,\N,87.0,12951.0,False
7955,9981,tt9741310,Slaxx,null,2020,\N,77.0,2464.0,False
7956,9982,tt9742392,Kindred,null,2020,\N,101.0,1719.0,False
7957,9996,tt9850386,The Bee Gees: How Can You Mend a Broken Heart,None,2020,\N,111.0,4144.0,True


In [ ]:
#We will put endYear as 2022 and create a new feature yeardif = endYear-startYear, and test for correlation corr().
#We'll do the same with every new feature we put to see if it actually helps lol.